In [1]:
import numpy as np
import pandas as pd
import numpy as np
from itertools import cycle
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_curve, f1_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import ADASYN

# Any results you write to the current directory are saved as output.

In [2]:
#load training data
data = pd.read_csv('train.csv')
data.head(5)

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38312 entries, 0 to 38311
Data columns (total 19 columns):
EmployeeNo                             38312 non-null object
Division                               38312 non-null object
Qualification                          36633 non-null object
Gender                                 38312 non-null object
Channel_of_Recruitment                 38312 non-null object
Trainings_Attended                     38312 non-null int64
Year_of_birth                          38312 non-null int64
Last_performance_score                 38312 non-null float64
Year_of_recruitment                    38312 non-null int64
Targets_met                            38312 non-null int64
Previous_Award                         38312 non-null int64
Training_score_average                 38312 non-null int64
State_Of_Origin                        38312 non-null object
Foreign_schooled                       38312 non-null object
Marital_Status                         383

In [4]:
# Create dummy variables for every unique category in Division
le = LabelEncoder()
data["Division"] = le.fit_transform(data["Division"])
data["Gender"] = le.fit_transform(data["Gender"])
data["Channel_of_Recruitment"] = le.fit_transform(data["Channel_of_Recruitment"])
data["Foreign_schooled"] = le.fit_transform(data["Foreign_schooled"])
data["Marital_Status"] = le.fit_transform(data["Marital_Status"])
data["Past_Disciplinary_Action"] = le.fit_transform(data["Past_Disciplinary_Action"])
data["State_Of_Origin"] = le.fit_transform(data["State_Of_Origin"])
data["Previous_IntraDepartmental_Movement"] = le.fit_transform(data["Previous_IntraDepartmental_Movement"])

In [5]:
data['No_of_previous_employers'] = data['No_of_previous_employers'].replace('More than 5', 6)
data['No_of_previous_employers'] = pd.to_numeric(data['No_of_previous_employers'])
data.drop(['Qualification', 'EmployeeNo'], axis=1, inplace=True)
data.head(5)

,Division,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,1,0,1,2,1986,12.5,2011,1,0,41,3,0,0,0,0,0,0
1,2,1,0,2,1991,12.5,2015,0,0,52,3,1,0,0,0,0,0
2,1,1,1,2,1987,7.5,2012,0,0,42,20,1,0,0,0,0,0
3,1,1,0,3,1982,2.5,2009,0,0,42,26,1,2,0,0,1,0
4,4,1,1,3,1990,7.5,2012,0,0,77,2,1,0,0,0,1,0


In [6]:
#split training dataset into train and test
y = data['Promoted_or_Not']
X = data.drop('Promoted_or_Not', axis=1)

In [7]:
# get class distribution
print ("Not Promoted:", data['Promoted_or_Not'][data['Promoted_or_Not'] == 0].count()) #class = 0
print ("Promoted:", data['Promoted_or_Not'][data['Promoted_or_Not'] == 1].count()) #class = 1

Not Promoted: 35071
Promoted: 3241


In [8]:
# Oversampling the dataset
sm = ADASYN(random_state=1)
X, y = sm.fit_sample(X, y)
# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 
          'nthread':[4],
          'gamma':[i/10.0 for i in range(3,6)],  
          'learning_rate': [.03, 0.05, .07, 0.1],
          'subsample':[i/10.0 for i in range(6,11)],
          'min_child_weight': [4],
          'silent': [1],
          'colsample_bytree':[i/10.0 for i in range(6,11)], 
          'max_depth': [2,3,4],
          'n_estimators': [1000]}

XGB_model = XGBClassifier(objective ='binary:hinge', 
                                         silent=False,
                                         scale_pos_weight=1,
                                         learning_rate = 0.01,
                                         subsample = 1.0,
                                         n_estimators=400, 
                                         max_depth=16, 
                                         booster = 'dart',
                                         colsample_bytree = 0.6,
                                         reg_alpha = 0.28,
                                         gamma=0.4,
                                         verbosity=2)
# XGB_model = RandomizedSearchCV(XGB_model, params, n_jobs=1,n_iter=10)
XGB_model.fit(X_train, y_train)

[16:01:04] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1576 extra nodes, 50 pruned nodes, max_depth=16
[16:01:04] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:01:04] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1510 extra nodes, 68 pruned nodes, max_depth=16
[16:01:04] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:01:05] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1536 extra nodes, 18 pruned nodes, max_depth=16
[16:01:05] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:01:05] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1934 extra nodes, 76 pruned n

[16:01:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1246 extra nodes, 78 pruned nodes, max_depth=16
[16:01:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:01:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2374 extra nodes, 80 pruned nodes, max_depth=16
[16:01:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:01:20] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1906 extra nodes, 36 pruned nodes, max_depth=16
[16:01:20] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:01:21] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1886 extra nodes, 102 pruned 

[16:01:45] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 2146 extra nodes, 40 pruned nodes, max_depth=16
[16:01:45] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:01:47] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1128 extra nodes, 14 pruned nodes, max_depth=16
[16:01:47] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:01:48] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1136 extra nodes, 74 pruned nodes, max_depth=16
[16:01:48] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:01:49] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1136 extra nodes, 32 pruned n

[16:02:31] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1326 extra nodes, 24 pruned nodes, max_depth=16
[16:02:31] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:02:32] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 926 extra nodes, 62 pruned nodes, max_depth=16
[16:02:32] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:02:34] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1370 extra nodes, 92 pruned nodes, max_depth=16
[16:02:34] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:02:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1978 extra nodes, 78 pruned no

[16:03:39] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1440 extra nodes, 66 pruned nodes, max_depth=16
[16:03:39] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:03:41] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1204 extra nodes, 44 pruned nodes, max_depth=16
[16:03:41] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:03:44] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1178 extra nodes, 56 pruned nodes, max_depth=16
[16:03:44] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:03:47] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1234 extra nodes, 34 pruned n

...

[16:23:57] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 912 extra nodes, 240 pruned nodes, max_depth=16
[16:23:57] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:24:09] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1736 extra nodes, 22 pruned nodes, max_depth=16
[16:24:09] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:24:22] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1674 extra nodes, 20 pruned nodes, max_depth=16
[16:24:22] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:24:34] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1938 extra nodes, 96 pruned n

[16:30:34] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1636 extra nodes, 60 pruned nodes, max_depth=16
[16:30:34] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:30:49] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 764 extra nodes, 316 pruned nodes, max_depth=16
[16:30:49] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:31:04] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1136 extra nodes, 166 pruned nodes, max_depth=16
[16:31:04] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:31:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1828 extra nodes, 74 pruned 

[16:38:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1352 extra nodes, 46 pruned nodes, max_depth=16
[16:38:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:38:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 824 extra nodes, 260 pruned nodes, max_depth=16
[16:38:36] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:38:54] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1328 extra nodes, 40 pruned nodes, max_depth=16
[16:38:54] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:39:12] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1332 extra nodes, 86 pruned n

[16:47:27] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1520 extra nodes, 68 pruned nodes, max_depth=16
[16:47:27] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:47:47] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1164 extra nodes, 88 pruned nodes, max_depth=16
[16:47:47] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:48:08] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 994 extra nodes, 24 pruned nodes, max_depth=16
[16:48:08] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\gbm\gbtree.cc:479: drop 0 trees, weight = 1
[16:48:29] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 1538 extra nodes, 104 pruned n

XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0.4,
              learning_rate=0.01, max_delta_step=0, max_depth=16,
              min_child_weight=1, missing=None, n_estimators=400, n_jobs=1,
              nthread=None, objective='binary:hinge', random_state=0,
              reg_alpha=0.28, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=False, subsample=1.0, verbosity=2)

In [9]:
XGB_predictions = XGB_model.predict(X_test)
XGB_predictions = [round(value) for value in XGB_predictions]
score = f1_score(y_test, XGB_predictions)
print(score)

0.9679177623715036


In [10]:
#load and predict using the test dataset provided
test_id = pd.read_csv('test.csv')
test = pd.read_csv('test.csv')

test["Division"] = le.fit_transform(test["Division"])
test["Gender"] = le.fit_transform(test["Gender"])
test["Channel_of_Recruitment"] = le.fit_transform(test["Channel_of_Recruitment"])
test["Foreign_schooled"] = le.fit_transform(test["Foreign_schooled"])
test["Marital_Status"] = le.fit_transform(test["Marital_Status"])
test["Past_Disciplinary_Action"] = le.fit_transform(test["Past_Disciplinary_Action"])
test["State_Of_Origin"] = le.fit_transform(test["State_Of_Origin"])
test['No_of_previous_employers'] = test['No_of_previous_employers'].replace('More than 5', 6)
test["Previous_IntraDepartmental_Movement"] = le.fit_transform(test["Previous_IntraDepartmental_Movement"])
test['No_of_previous_employers'] = pd.to_numeric(test['No_of_previous_employers'])
test.drop(['Qualification', 'EmployeeNo'], axis=1, inplace=True)

pred = XGB_model.predict(test.values)
pred = [int(round(value)) for value in pred]

output = pd.DataFrame({'EmployeeNo': test_id.EmployeeNo, 'Promoted_or_Not': pred})
output.to_csv('staffCVsubmission.csv', index=False)

In [11]:
test.shape

(16496, 16)